In [1]:
%reset -f

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline, GPTQConfig, pipeline
from auto_gptq import exllama_set_max_input_length

import platform
import os
import shutil

import spacy

from math import ceil
from spacy.lang.en import English 

In [3]:
if platform.system() != 'Linux':
    
    print("GPTQ is optimized for NV cards on cuda-enabled Linux. Please check your OS")

else :
    
    os.chdir("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp")
    print("Linux detected.")
    print("the current wd is ", os.getcwd())


Linux detected.
the current wd is  /home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp


In [4]:
model_name_or_path = "/home/ning/workdir/model/wd-v0.0.1"
# model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
# To use a different branch, change revision
# For example: revision="main"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="gptq-4bit-128g-actorder_True")

# model = exllama_set_max_input_length(model, 2048)

# model.save_pretrained("/home/ning/workdir/model/docsum-testmodel-v0.0.1", from_pt=True) 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
start_wrd = "Summary:"

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-GPTQ", use_fast=True)

# sliding windows : evaluating prompts $\geqslant$ 4096 tokens

In [6]:
with open("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/data/feature/test1.txt") as f:
    doc = f.read()

nlp = spacy.load("en_core_web_sm") 

result = nlp(doc)

window_size = 1000
grid_size = 2000
shards_number = ceil(len(result)/grid_size)
file_shards = [None] * shards_number

for i in range(shards_number):

    tmp = result[i *grid_size : i * grid_size + window_size]
    file_shards[i] = ''.join([token.text_with_ws for token in tmp])

    with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/data/feature/test1_' + str(i) + '.txt', 'w', encoding="utf-8") as f:
        f.write(file_shards[i])

In [7]:
short_sum = []

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1
    )

for i in range(shards_number):

    with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/data/feature/test1_' + str(i) + '.txt', 'r+', encoding="utf-8") as file:
        context = file.read()



    prompt_template=f'''You are a helpful and patient AI assistant. You always provide all the details. Take a deep breathe and write a summary step by step about the following artile. Include all the details. \n
    Article: {short_sum} \n {context} \n 
    Summary: 
    '''

    result = pipe(prompt_template)

    raw_short_result = result[0]['generated_text']
    short_sum_idx = raw_short_result.find(start_wrd)
    short_sum = raw_short_result[short_sum_idx+8:]

with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/result/cumulated_sum.txt', 'w', encoding="utf-8") as f:
    f.write(short_sum)

print(short_sum)

 
    
Hami Qutami was initially sentenced to 4 years and 5 months, with an 11 months and 21 days non-parole period for trying to hire someone to kill his niece Ruba due to cultural and religious disagreements. He challenged the decision, claiming that the sentence was far too lenient and the judge made mistakes when determining the non-parole period.

The prosecution contended that reducing the non-parole period was unwarranted, despite acknowledging Qutami's good behavior and health. Nevertheless, they also maintained that the punishment was excessively lenient and failed to reflect the severity of the crime.

Qutami, on the other hand, insisted that the sentence was fair and that the judge had not made any errors. He highlighted the importance of peace within communities and his medical issues.

The court eventually found several problems with the initial ruling, declaring that the non-parole period was too brief and failed to adequately reflect the severity of the crime. Additional

# evaluating prompts $\leqslant$ 4096 tokens

In [8]:
with open("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/data/feature/os-1.txt") as f:
    context = f.read()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

short_prompt_template=f'''You are a helpful and patient AI assistant. You always provide all the details. Take a deep breathe and write a long summary step by step about the following artile. Include all the details. \n
 Article: {context} \n 
 Summary: 
'''

result = pipe(short_prompt_template)

raw_short_result = result[0]['generated_text']
short_sum_idx = raw_short_result.find(start_wrd)
short_sum = raw_short_result[short_sum_idx+8:]
print(short_sum)

with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/result/short_sum.txt', 'w', encoding="utf-8") as f:
    f.write(short_sum)

 

This is a medical report for Mr Tan Ah Kow, a 55-year-old male patient. He has a history of various medical conditions including hypertension, hyperlipidemia, stroke, cardiomyopathy, cardiac failure, and chronic renal disease. He has been under the care of Dr. Tan Ah Moi since November 2010. During a recent examination on 20th June 2015, he displayed signs of cognitive impairment and was unable to recall important information or perform simple tasks. He was diagnosed with Dementia and stroke based on these observations. The doctor opines that Mr Tan's cognitive functions are unlikely to improve and may deteriorate further with time.
